In [5]:
from Environment import Ecosystem

In [6]:
default_params = {
  "mapsize": 1024,
  "min_depth": 10,
  "max_depth": 20,
  "starting_population": 10,
  "food_value": 200,
  "max_timesteps": 200000,
  "movement_cost_factor": 0.005,
  "food_per_agent": 4,
  "egg_incubation_time": 100,
  "mutation_factor": 2
}

In [7]:
env = Ecosystem("human", params=default_params, debug=False)
env.reset()
for i in range(10000000):
  env.stepper()
  if len(env.agentNames) == 0:
    break
env.close()

[[10 10 10 ... 11 11 11]
 [10 10 10 ... 11 11 11]
 [10 10 10 ... 11 11 11]
 ...
 [12 12 12 ... 20 20 20]
 [12 12 12 ... 20 20 20]
 [12 12 12 ... 20 20 20]]
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agent death
Agen

In [8]:
env.close()

Closing environment Ecosystem at 6152 timestep.
